In [2]:
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2

Using TensorFlow backend.


In [3]:
def cv_disp_img(img, title="image"):
    while True:
        cv2.imshow(title, img)
        k = cv2.waitKey(1)

        if k & 0xFF == ord('q'):
            # q key pressed so quit
            print("Quitting...")
            break

    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [9]:
features = np.load("./PP_Data/features.npy")
labels = np.load("./PP_Data/labels.npy")

test_features = np.load("./PP_Data/test_features.npy")
test_labels = np.load("./PP_Data/test_labels.npy")

X_train = features
y_train = labels
X_val = test_features
y_val = test_labels
n_input = features.shape[1]
n_class = labels.shape[1]
batch_size = X_train.shape[0]//10
X_train.shape,y_train.shape,batch_size

((680, 225), (680, 4), 68)

In [10]:
n_class,n_input

(4, 225)

In [6]:
def build_keras_base(hidden_layers = [50], dropout_rate = 0, 
                     l2_penalty = 0.1, optimizer = 'adam',
                     n_input = n_input, n_class = n_class):
    """
    Keras Multi-layer neural network. Fixed parameters include: 
    1. activation function (PRelu)
    2. always uses batch normalization after the activation
    3. use adam as the optimizer
    
    Parameters
    ----------
    Tunable parameters are (commonly tuned)
    
    hidden_layers: list
        the number of hidden layers, and the size of each hidden layer
    
    dropout_rate: float 0 ~ 1
        if bigger than 0, there will be a dropout layer
    
    l2_penalty: float
        or so called l2 regularization
    
    optimizer: string or keras optimizer
        method to train the network
    
    Returns
    -------
    model : 
        a keras model

    Reference
    ---------
    https://keras.io/scikit-learn-api/
    """   
    model = Sequential()   
    for index, layers in enumerate(hidden_layers):       
        if not index:
            # specify the input_dim to be the number of features for the first layer
            model.add(Dense(layers, input_dim = n_input, kernel_regularizer = l2(l2_penalty)))
        else:
            model.add(Dense(layers, kernel_regularizer = l2(l2_penalty)))
        
        # insert BatchNorm layer immediately after fully connected layers
        # and before activation layer
        model.add(BatchNormalization())
        model.add(PReLU())        
        if dropout_rate:
            model.add(Dropout(p = dropout_rate))
    
    model.add(Dense(n_class))
    model.add(Activation('softmax'))
    
    # the loss for binary and muti-class classification is different 
    loss = 'binary_crossentropy'
    if n_class > 2:
        loss = 'categorical_crossentropy'
    
    model.compile(loss = loss, optimizer = optimizer, metrics = ['accuracy'])   
    return model

In [11]:
# pass in fixed parameters n_input and n_class
model_keras = KerasClassifier(
    build_fn = build_keras_base,
    n_input = n_input,
    n_class = n_class,
)

# random search's parameter:
# specify the options and store them inside the dictionary
# batch size and training method can also be hyperparameters, 
# but it is fixed
early_stop = EarlyStopping(
    monitor = 'val_loss', min_delta = 0.1, patience = 5, verbose = 0)

callbacks = [early_stop]
keras_fit_params = {   
    'callbacks': callbacks,
    'epochs': 100,
    'batch_size': batch_size,
    'validation_data': {'input': X_val, 
                        'output': y_val},
    'verbose': 0
}

dropout_rate_opts  = [0, 0.2, 0.5]
hidden_layers_opts = []
for i in range(1,800):
    hidden_layers_opts.append(tuple((i,)))
    for j in range(1,1000):
        hidden_layers_opts.append(tuple((i,j)))
#         for k in range(1,300):
#             hidden_layers_opts.append(tuple((i,j,k)))
# hidden_layers_opts = [(, 2, 64, 64), (32, 32, 32, 32, 32), (100, 100, 100)]
l2_penalty_opts = [0.01, 0.1, 0.5]
keras_param_options = {
    'hidden_layers': hidden_layers_opts,
    'dropout_rate': dropout_rate_opts,  
    'l2_penalty': l2_penalty_opts,
    
}

In [12]:
# `verbose` 2 will print the class info for every cross validation, 
# kind of too much
rs_keras = RandomizedSearchCV( 
    model_keras, 
    param_distributions = keras_param_options,
    scoring = 'neg_log_loss',
    cv=5,
    n_jobs = -1,
    verbose = 10
)
rs_keras.fit(X_train, y_train,
             validation_data = (X_val, y_val),
            callbacks=callbacks,
            epochs=100,
            batch_size=batch_size,
            verbose=1)

print('Best score obtained: {0}'.format(rs_keras.best_score_))
print('Parameters:')
for param, value in rs_keras.best_params_.items():
    print('\t{}: {}'.format(param, value))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   21.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   43.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:  4.4min remaining:   57.7s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:  4.9min remaining:   18.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  4.9min finished
/Users/connordepalma/anaconda3/envs/CPE646/lib/python3.7/site-packages/ipykernel_launcher.py:48: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`



Train on 680 samples, validate on 23 samples
Epoch 1/100
680/680 [==============================] - 3s 4ms/step - loss: 9.9731 - accuracy: 0.6824 - val_loss: 45.4745 - val_accuracy: 0.2174
Epoch 2/100
680/680 [==============================] - 0s 460us/step - loss: 8.3115 - accuracy: 0.9147 - val_loss: 32.0866 - val_accuracy: 0.2174
Epoch 3/100
680/680 [==============================] - 0s 494us/step - loss: 7.0348 - accuracy: 0.9691 - val_loss: 22.1564 - val_accuracy: 0.2174
Epoch 4/100
680/680 [==============================] - 0s 492us/step - loss: 5.8817 - accuracy: 0.9809 - val_loss: 16.9881 - val_accuracy: 0.2174
Epoch 5/100
680/680 [==============================] - 0s 498us/step - loss: 4.8626 - accuracy: 0.9882 - val_loss: 12.3628 - val_accuracy: 0.2174
Epoch 6/100
680/680 [==============================] - 0s 477us/step - loss: 3.9669 - accuracy: 0.9941 - val_loss: 8.2684 - val_accuracy: 0.2609
Epoch 7/100
680/680 [==============================] - 0s 459us/step - loss: 3.24

In [13]:
l2_pen = rs_keras.best_params_["l2_penalty"]
hl = rs_keras.best_params_["hidden_layers"]
dr = rs_keras.best_params_["dropout_rate"]
l2_pen,hl,dr

(0.01, (768, 499), 0.5)

In [49]:
(loss, accuracy) = rs_keras.best_estimator_.model.evaluate(X_val, y_val,
    batch_size=5, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,
    accuracy * 100))

# dump the network architecture and weights to file
print("[INFO] dumping architecture and weights to file...")
rs_keras.best_estimator_.model.save("./model.hdf5")

23/23 [==============================] - 0s 662us/step
[INFO] loss=0.3903, accuracy: 95.6522%
[INFO] dumping architecture and weights to file...
